# Entraînement aux Réseaux Antagonistes Génératifs 
## TRINCKLIN Paul
## SEVESTRE Vincent

Dans ce notebook, un modèle de GAN est créé de toutes pièces. Il est mis à exécution pour générer des images de "Pokemon". Plus de 800 images sont utilisées pour entraîner le modèle.

## Traîtement des données

### Recadrage des images

In [55]:
import os
import cv2

src = "./processing_data/data_raw" #imagesRGB brutes
dst = "./processing_data/resizedData" # images RGB remises à la bonne taille

os.mkdir(dst)

for each in os.listdir(src):
    img = cv2.imread(os.path.join(src,each))
    img = cv2.resize(img,(256,256))
    cv2.imwrite(os.path.join(dst,each), img)

### Transformation du code couleur

In [57]:
from PIL import Image

src = "./processing_data/resizedData"
dst = "./processing_data/resized_black/"

for each in os.listdir(src):
    png = Image.open(os.path.join(src,each))
    # print each
    if png.mode == 'RGBA':
        png.load() # required for png.split()
        background = Image.new("RGB", png.size, (0,0,0))
        background.paste(png, mask=png.split()[3]) # 3 est le canal alpha
        background.save(os.path.join(dst,each.split('.')[0] + '.jpg'), 'JPEG')
    else:
        png.convert('RGB')
        png.save(os.path.join(dst,each.split('.')[0] + '.jpg'), 'JPEG')


## Construction du GAN

In [58]:
import os
import tensorflow as tf
import numpy as np
import cv2
import random
import scipy.misc
from utils import *
import warnings
import time
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Conv2DTranspose, Reshape
from tensorflow.keras.layers import Flatten, BatchNormalization, Dense, Activation
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Générateur

Le générateur fonctione comme une sorte de réseau neuronal convolutif inverse. Un réseau neuronal convolutif typique, comme le réseau du discriminateur, transforme une matrice bidimensionnelle ou tridimensionnelle de valeurs de pixels en une probabilité unique. Un générateur, en revanche, prend un vecteur de bruit à d dimensions et le suréchantillonne pour en faire une image de 64 x 64 pixels.

In [59]:
def get_generator():
    """
    Crée le réseau du générateur
    """
    generator = Sequential()
    generator.add(Dense(units=4 * 4 * 512,
                        kernel_initializer='glorot_uniform',
                        input_shape=(1, 1, 100)))
    generator.add(Reshape(target_shape=(4, 4, 512)))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=256, kernel_size=(5, 5),
                                    strides=(2, 2), padding='same',
                                    data_format='channels_last',
                                    kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=128, kernel_size=(5, 5),
                                    strides=(2, 2), padding='same',
                                    data_format='channels_last',
                                    kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=64, kernel_size=(5, 5),
                                    strides=(2, 2), padding='same',
                                    data_format='channels_last',
                                    kernel_initializer='glorot_uniform'))
    generator.add(BatchNormalization(momentum=0.5))
    generator.add(Activation('relu'))

    generator.add(Conv2DTranspose(filters=3, kernel_size=(5, 5),
                                    strides=(2, 2), padding='same',
                                    data_format='channels_last',
                                    kernel_initializer='glorot_uniform'))
    generator.add(Activation('tanh'))

    optimizer = Adam(learning_rate=0.00015, beta_1=0.5)
    generator.compile(loss='binary_crossentropy',
                        optimizer=optimizer,
                        metrics=None)

    return generator

### Discriminateur

Le discriminateur est un réseau neuronal convolutif qui prend en entrée une image de taille 64 x 64 et renvoie un nombre scalaire unique décrivant si l'image d'entrée est "réelle" ou "fausse"

In [60]:
def get_discriminator(image_shape):
        """
        Crée le réseau du discriminateur
        """
        discriminator = Sequential()
        discriminator.add(Conv2D(filters=64, kernel_size=(5, 5),
                                 strides=(2, 2), padding='same',
                                 data_format='channels_last',
                                 kernel_initializer='glorot_uniform',
                                 input_shape=image_shape))
        discriminator.add(LeakyReLU(0.2))

        discriminator.add(Conv2D(filters=128, kernel_size=(5, 5),
                                 strides=(2, 2), padding='same',
                                 data_format='channels_last',
                                 kernel_initializer='glorot_uniform'))
        discriminator.add(BatchNormalization(momentum=0.5))
        discriminator.add(LeakyReLU(0.2))

        discriminator.add(Conv2D(filters=256, kernel_size=(5, 5),
                                 strides=(2, 2), padding='same',
                                 data_format='channels_last',
                                 kernel_initializer='glorot_uniform'))
        discriminator.add(BatchNormalization(momentum=0.5))
        discriminator.add(LeakyReLU(0.2))

        discriminator.add(Conv2D(filters=512, kernel_size=(5, 5),
                                 strides=(2, 2), padding='same',
                                 data_format='channels_last',
                                 kernel_initializer='glorot_uniform'))
        discriminator.add(BatchNormalization(momentum=0.5))
        discriminator.add(LeakyReLU(0.2))

        discriminator.add(Flatten())
        discriminator.add(Dense(1))
        discriminator.add(Activation('sigmoid'))

        optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
        discriminator.compile(loss='binary_crossentropy',
                              optimizer=optimizer,
                              metrics=None)

        return discriminator

### Modèle complet

In [61]:
def get_adversarial(generator, discriminator):
        """
        Construit le réseau complet
        """
        gan = Sequential()
        discriminator.trainable = False
        gan.add(generator)
        gan.add(discriminator)

        optimizer = Adam(learning_rate=0.00015, beta_1=0.5)
        gan.compile(loss='binary_crossentropy', optimizer=optimizer,
                    metrics=None)
        return gan

### Data loaders / savers

In [62]:
def load_data(dataset_path, image_shape, batch_size):
    """
    Charge les données à partir de ce qui est présent dans "dataset_path"
    """
    image_data_generator = ImageDataGenerator()
    dataset_generator = image_data_generator.flow_from_directory(
        dataset_path, target_size=(image_shape[0], image_shape[1]),
        batch_size=batch_size,
        class_mode=None)
    return dataset_generator

In [63]:
def save_images(generated_images, epoch_no, batch_no):
        """
        Affiche et sauvegarde les images
        """
        print("Saving generated images to new_images/")
        plt.figure(figsize=(8, 8), num=2)
        gs1 = gridspec.GridSpec(8, 8)
        gs1.update(wspace=0, hspace=0)

        for i in range(64):
            ax1 = plt.subplot(gs1[i])
            ax1.set_aspect('equal')
            image = generated_images[i, :, :, :]
            image += 1
            image *= 127.5
            fig = plt.imshow(image.astype(np.uint8))
            plt.axis('off')
            fig.axes.get_xaxis().set_visible(False)
            fig.axes.get_yaxis().set_visible(False)

        plt.tight_layout()
        save_name = 'new_images/generated_epoch' + str(
            epoch_no + 1) + '_batch' + str(batch_no + 1) + '.png'
        if not os.path.exists('new_images'):
            os.mkdir('new_images')
        plt.savefig(save_name, bbox_inches='tight', pad_inches=0)
        plt.pause(0.0000000001)
        plt.show()

### Séquence d'entraînement

In [64]:
def train_model(dataset_path, image_shape, batch_size, epochs):
    """
    Entraîne le GAN, et sauvegarde les images générées à chaque étape
    """
    generator = get_generator()
    discriminator = get_discriminator(image_shape)
    gan = get_adversarial(generator, discriminator)

    # Load dataset
    dataset_generator = load_data(dataset_path, image_shape, batch_size)

    number_of_batches = 150

    # Ces variables vont conserver les pertes des deux modèles
    adversarial_loss = np.empty(shape=1)
    discriminator_loss = np.empty(shape=1)
    batches = np.empty(shape=1)

    # Pour pouvoir plot dans les boucles
    plt.ion()

    current_batch = 0

    # Début de l'entraînement
    for epoch in range(epochs):

        print("Epoch " + str(epoch + 1) + "/" + str(epochs) + " :")

        for batch_number in range(number_of_batches):

            start_time = time.time()

            real_images = dataset_generator.next()

            # Normalisation des vecteurs images
            real_images /= 127.5
            real_images -= 1

            # Étant donné que le dernier batch n'est pas de la même taille que les autres, 
            # il faut prendre en compte la taille
            current_batch_size = real_images.shape[0]

            # Création de bruit, de façon aléatoire
            noise = np.random.normal(0, 1,
                                        size=(current_batch_size,) + (1, 1, 100))

            # Génération d'images par un simple passage dans le générateur
            generated_images = generator.predict(noise)

            # Ajout de bruit pour les labels qui seront donnés au discriminateur
            real_y = (np.ones(current_batch_size) -
                        np.random.random_sample(current_batch_size) * 0.2)
            fake_y = np.random.random_sample(current_batch_size) * 0.2

            # Entraînement du discriminateur
            discriminator.trainable = True

            d_loss = discriminator.train_on_batch(real_images, real_y)
            d_loss += discriminator.train_on_batch(generated_images, fake_y)

            discriminator_loss = np.append(discriminator_loss, d_loss)

            # Puis entraînement du générateur
            discriminator.trainable = False
            print("Taille batch actuelle :", current_batch_size)
            noise = np.random.normal(0, 1,
                                        size=(current_batch_size * 2,) +
                                            (1, 1, 100))

            # On reprend la formule précédente de bruit, en inversant les labels pour tromper le discriminateur
            # ici la formule correspond à celle de "real_y" plus haut
            fake_y = (np.ones(current_batch_size * 2) -
                        np.random.random_sample(current_batch_size * 2) * 0.2)

            g_loss = gan.train_on_batch(noise, fake_y)
            adversarial_loss = np.append(adversarial_loss, g_loss)
            batches = np.append(batches, current_batch)

            # Sauvegarde d'image tous les 50 batchs
            if ((batch_number + 1) % 50 == 0 and
                    current_batch_size == batch_size):
                save_images(generated_images, epoch, batch_number)

            time_elapsed = time.time() - start_time

            # Affichage des valeurs de pertes
            print("     Batch " + str(batch_number + 1) + "/" +
                    str(number_of_batches) +
                    " Perte générateur | Perte discriminateur : " +
                    str(g_loss) + " | " + str(d_loss) + ' - Temps batch ' +
                    str(time_elapsed) + ' s.')

            current_batch += 1

        # Sauvegarde des poids du modèle tous les 5 epochs
        if (epoch + 1) % 5 == 0:
            discriminator.trainable = True
            if not os.path.exists('models'):
                os.mkdir('models')
            generator.save('models/generator_epoch' + str(epoch) + '.hdf5')
            discriminator.save('models/discriminator_epoch' +
                                str(epoch) + '.hdf5')

        # A chaque époch, on traçe les courbes de pertes
        plt.figure(1)
        plt.plot(batches, adversarial_loss, color='green',
                    label='Perte Générateur')
        plt.plot(batches, discriminator_loss, color='blue',
                    label='Perte Discriminateur')
        plt.title("Entraînement GAN")
        plt.xlabel("Itération Batch")
        plt.ylabel("Perte")
        plt.legend()
        plt.pause(0.0000000001)
        plt.show()
        plt.savefig('trainingLossPlot.png')

In [ ]:
dataset_path = './data/'
batch_size = 64
image_shape = (64, 64, 3)
epochs = 45
train_model(dataset_path, image_shape, batch_size, epochs)

## Traitement des résultats

In [72]:
# Clip gif du débruitage 
import moviepy.editor as mpy
clip = mpy.ImageSequenceClip(sequence="new_images/", fps=6)
clip.write_gif('denoising_6.gif')

MoviePy - Building file denoising_6.gif with imageio.


<img src="denoising_10.gif" width="450" align="center">